### Can we classify each phase as relatively low or high anxiety for each subject? ###

In [2]:
# IMPORTING MODULES
import glob
import importlib
import matplotlib.pyplot as plt
import numpy as np
import os
cvx_path = os.path.abspath(os.path.join('..', '..', 'cvxEDA', 'src'))
module_path = os.path.abspath(os.path.join('..', '..', 'src'))
import pandas as pd
import random
import scipy.signal as ss
import shap
import sys
sys.path.append(module_path)

import tools.data_reader_apd as dr
import tools.display_tools as dt
import tools.preprocessing as preprocessing
import train

import lightgbm as lgb
from lightgbm import LGBMClassifier
from scipy.fft import fft, fftfreq, fftshift
from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import GaussianNB
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix, \
    mean_absolute_error, mean_squared_error, log_loss
from sklearn.model_selection import train_test_split, cross_validate, RepeatedKFold
from sklearn.neighbors import KNeighborsClassifier
from sklearn.preprocessing import normalize
from xgboost import XGBClassifier

import cvxopt.solvers
cvxopt.solvers.options['show_progress'] = False

import warnings
warnings.filterwarnings(
    "ignore", 
    category=RuntimeWarning
)

temp_a, _ = train.Train_APD.get_apd_data_ranking([train.Metrics.BPM], phases=dr.Phases.PHASES_LIST, normalize=False)
idx = temp_a[temp_a["bpm"] > 200].index 
invalid_apd_subjects = set(temp_a["subject"].iloc[idx].tolist())
idx = temp_a[temp_a["bpm"] < 35].index 
invalid_apd_subjects.update(set(temp_a["subject"].iloc[idx].tolist()))

c:\Users\zhoux\AppData\Local\Programs\Python\Python37\lib\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [30]:
importlib.reload(train)

models = {
    "SVM": SVC(),
    "LGB": LGBMClassifier(),
    "RF": RandomForestClassifier(random_state=16),
    "XGB": XGBClassifier(random_state=16),
    # "random": None
}

parameters = {
    "SVM": [{
        "kernel": ["rbf", "poly", "sigmoid"],
        "C": [0.1, 1, 10, 100],
        "gamma": [1, 0.1, 0.01, 0.001, "scale", "auto"],
    }],
    "LGB": [{
        "objective": ["binary"],
        "num_leaves": [10, 20, 30, 40, 50],
        "max_depth": [3, 4, 5, 6, 7],
        "metric": ["mean_absolute_error", "mean_squared_error", "binary_logloss"]
    }],
    "RF": [{
        "n_estimators": [10, 20, 30, 40, 50],
        "max_features": ["sqrt", "0.4"],
        "min_samples_split": [3, 4, 5, 6, 7],
        "random_state": [16]
    }],
    "XGB": [{
        "objective": ["binary:logistic"],
        "learning_rate": [0.01, 0.1, 0.3, 0.5],
        "max_depth": [4, 5, 6, 7],
        "n_estimators": [10, 20, 30, 40, 50],
        "eval_metric": ["error"],
        "use_label_encoder": [False],
        "random_state": [16]
    }],
    # "random": None
}

metrics = [
    train.Metrics.BPM, 
    train.Metrics.RMSSD, 
    train.Metrics.HF_RR, 
    train.Metrics.LF_RR, 
    train.Metrics.SDNN, 
    train.Metrics.MEAN_SCL, 
    train.Metrics.SCR_RATE, 
]
# ] + train.Metrics.STATISTICAL

In [41]:
# K-FOLD CROSS-VALIDATION FOR HYPERPARAMETER SELECTION
importlib.reload(train)
importlib.reload(dr)
importlib.reload(dt)


random.seed(29)

model_phases = [
    "Baseline_Rest", 
    "BugBox_Relax", "BugBox_Anticipate", "BugBox_Exposure", "BugBox_Break",
    "Speech_Relax", "Speech_Anticipate", "Speech_Exposure", "Speech_Break"
]

threshold = "fixed"

anxiety_label_type = None
x, y = train.Train_APD.get_apd_data_ranking(
    metrics, model_phases, verbose=False, anxiety_label_type=anxiety_label_type, 
    threshold=threshold, normalize=True, combine_phases=False, standardize=False
)
x = x.drop(["phaseId"], axis=1)
inds = pd.isnull(x).any(axis=1).to_numpy().nonzero()[0]
x = x.drop(labels=inds, axis=0).reset_index(drop=True)
y = y.drop(labels=inds, axis=0).reset_index(drop=True)

# drop subjects with noisy data
# x = x[~x["subject"].isin(invalid_apd_subjects)].reset_index(drop=True)
# y = y[~y["subject"].isin(invalid_apd_subjects)].reset_index(drop=True)

if anxiety_label_type is not None:
    x.drop(labels=["anxietyGroup"], axis=1)
    
acc_results = {
    "SVM": [],
    "LGB": [],
    "RF": [],
    "XGB": [],
    # "random": []
}
reports = {
    "SVM": [],
    "LGB": [],
    "RF": [],
    "XGB": [],
    # "random": []
}
best_models = {}

num_iters = 5
get_importance = True
for _ in range(num_iters):
    # HYPERPARAMETER TUNING
    model_data = train.grid_search_cv(
        models, parameters, x, y, by_subject=True, save_metrics=True, is_resample=True,
        get_importance=get_importance, drop_subject=True, test_size=0.1, folds=5
    )

    for model_name in models.keys():
        best_models[model_name] = model_data[model_name]["best_model"]
        print(f"{model_name}: {model_data[model_name]['best_params']}")

    # # FEATURE SELECTION
    x_train, y_train = model_data["train"]
    features = {name: metrics+["lf_hf_ratio"] for name in models.keys()}
    # features = train.feature_selection(best_models, model_data["cv"], x_train, y_train, n_features=5)

    # TEST USING OPTIMIZED MODELS AND FEATURES
    x_test, y_test = model_data["test"]
    out = train.train_test_model(best_models, features, x_train, y_train, x_test, y_test)

    for model_name in acc_results:
        acc_results[model_name].append(out[model_name]["performance"][0])
        reports[model_name].append(out[model_name]["performance"][1])
        # if get_importance:
            # try:
                # print("")
                # feature_imp = list(zip(metrics + ["lf_hf_ratio"], out[model_name]["performance"][2]))
                # feature_imp = sorted(feature_imp, key=lambda x: x[1], reverse=True)
                # print(feature_imp)
            # except Exception as e:
                # print(out[model_name]["performance"][2])
            # print("")

for model_name in acc_results.keys():
    print(f"Model evaluation metrics for {model_name}:")
    for i in range(len(reports[model_name])):
        report = reports[model_name][i]
        acc = acc_results[model_name][i]
        p = report["precision"]
        r = report["recall"]
        f1 = report["f1"]
        auc = report["auc"]
        print(f"\tAccuracy: {acc}\n\tPrecision: {p}\n\tRecall: {r}\n\tF1-score: {f1}\n\tAUC score: {auc}\n" + "-"*40)
    print(f"Mean acc: {np.mean([acc_results[model_name][i] for i in range(len(reports[model_name]))])}")
    print(f"Mean F1-score: {np.mean([reports[model_name][i]['f1'] for i in range(len(reports[model_name]))])}")
    print(f"Mean AUC score: {np.mean([reports[model_name][i]['auc'] for i in range(len(reports[model_name]))])}")
    print("\n")

Grid search for SVM ...
Grid search for LGB ...
Grid search for RF ...


One or more of the test scores are non-finite: [0.54226638 0.5319911  0.54476925 0.52701917 0.53523926 0.53747928
 0.5276666  0.53533911 0.54026449 0.54635302 0.51703723 0.5266784
 0.53172681 0.52460581 0.53387102 0.50594257 0.51049549 0.52107168
 0.52710246 0.53210701 0.53918675 0.53633961 0.5356961  0.52959644
 0.5383409         nan        nan        nan        nan        nan
        nan        nan        nan        nan        nan        nan
        nan        nan        nan        nan        nan        nan
        nan        nan        nan        nan        nan        nan
        nan        nan]


Grid search for XGB ...
SVM: {'C': 100, 'gamma': 'auto', 'kernel': 'sigmoid'}
LGB: {'max_depth': 3, 'metric': 'mean_absolute_error', 'num_leaves': 10, 'objective': 'binary'}
RF: {'max_features': 'sqrt', 'min_samples_split': 4, 'n_estimators': 50, 'random_state': 16}
XGB: {'eval_metric': 'error', 'learning_rate': 0.01, 'max_depth': 6, 'n_estimators': 30, 'objective': 'binary:logistic', 'random_state': 16, 'use_label_encoder': False}
Training SVM ...
Model SVM, Actual: [0 1], [78 40], Predictions: [0 1], [54 64]
coef_ only available for SVC with linear kernel
Training LGB ...
Model LGB, Actual: [0 1], [78 40], Predictions: [0 1], [74 44]
Training RF ...
Model RF, Actual: [0 1], [78 40], Predictions: [0 1], [63 55]
Training XGB ...
Model XGB, Actual: [0 1], [78 40], Predictions: [0 1], [66 52]
Grid search for SVM ...
Grid search for LGB ...
Grid search for RF ...


One or more of the test scores are non-finite: [0.55514238 0.57094173 0.5585362  0.56263435 0.55492669 0.53487189
 0.53961219 0.53856036 0.54652851 0.54359344 0.54317866 0.55618339
 0.57026983 0.57149747 0.57020428 0.53878027 0.53604113 0.54009478
 0.53814154 0.53701604 0.54853183 0.54729021 0.54633985 0.55512254
 0.55584939        nan        nan        nan        nan        nan
        nan        nan        nan        nan        nan        nan
        nan        nan        nan        nan        nan        nan
        nan        nan        nan        nan        nan        nan
        nan        nan]


Grid search for XGB ...
SVM: {'C': 1, 'gamma': 0.01, 'kernel': 'sigmoid'}
LGB: {'max_depth': 7, 'metric': 'mean_absolute_error', 'num_leaves': 30, 'objective': 'binary'}
RF: {'max_features': 'sqrt', 'min_samples_split': 5, 'n_estimators': 40, 'random_state': 16}
XGB: {'eval_metric': 'error', 'learning_rate': 0.5, 'max_depth': 5, 'n_estimators': 10, 'objective': 'binary:logistic', 'random_state': 16, 'use_label_encoder': False}
Training SVM ...
Model SVM, Actual: [0 1], [64 52], Predictions: [0 1], [111   5]
coef_ only available for SVC with linear kernel
Training LGB ...
Model LGB, Actual: [0 1], [64 52], Predictions: [0 1], [53 63]
Training RF ...
Model RF, Actual: [0 1], [64 52], Predictions: [0 1], [46 70]
Training XGB ...
Model XGB, Actual: [0 1], [64 52], Predictions: [0 1], [43 73]
Grid search for SVM ...
Grid search for LGB ...
Grid search for RF ...


...

Grid search for XGB ...
SVM: {'C': 10, 'gamma': 0.1, 'kernel': 'rbf'}
LGB: {'max_depth': 7, 'metric': 'mean_absolute_error', 'num_leaves': 30, 'objective': 'binary'}
RF: {'max_features': 'sqrt', 'min_samples_split': 5, 'n_estimators': 50, 'random_state': 16}
XGB: {'eval_metric': 'error', 'learning_rate': 0.5, 'max_depth': 5, 'n_estimators': 50, 'objective': 'binary:logistic', 'random_state': 16, 'use_label_encoder': False}
Training SVM ...
Model SVM, Actual: [0 1], [78 36], Predictions: [0 1], [75 39]
coef_ only available for SVC with linear kernel
Training LGB ...
Model LGB, Actual: [0 1], [78 36], Predictions: [0 1], [64 50]
Training RF ...
Model RF, Actual: [0 1], [78 36], Predictions: [0 1], [63 51]
Training XGB ...
Model XGB, Actual: [0 1], [78 36], Predictions: [0 1], [63 51]
Grid search for SVM ...
Grid search for LGB ...
Grid search for RF ...


One or more of the test scores are non-finite: [0.55905239 0.58384067 0.58568131 0.58781393 0.59216269 0.55176586
 0.58291212 0.58466279 0.59135297 0.59061106 0.58649834 0.6010166
 0.60596748 0.60612052 0.60745702 0.57018686 0.59314855 0.60196257
 0.60891832 0.61332548 0.59739154 0.59908221 0.60059754 0.59823085
 0.60164763        nan        nan        nan        nan        nan
        nan        nan        nan        nan        nan        nan
        nan        nan        nan        nan        nan        nan
        nan        nan        nan        nan        nan        nan
        nan        nan]


Grid search for XGB ...
SVM: {'C': 100, 'gamma': 1, 'kernel': 'rbf'}
LGB: {'max_depth': 4, 'metric': 'mean_absolute_error', 'num_leaves': 20, 'objective': 'binary'}
RF: {'max_features': 'sqrt', 'min_samples_split': 6, 'n_estimators': 50, 'random_state': 16}
XGB: {'eval_metric': 'error', 'learning_rate': 0.3, 'max_depth': 4, 'n_estimators': 20, 'objective': 'binary:logistic', 'random_state': 16, 'use_label_encoder': False}
Training SVM ...
Model SVM, Actual: [0 1], [77 29], Predictions: [0 1], [46 60]
coef_ only available for SVC with linear kernel
Training LGB ...
Model LGB, Actual: [0 1], [77 29], Predictions: [0 1], [57 49]
Training RF ...
Model RF, Actual: [0 1], [77 29], Predictions: [0 1], [54 52]
Training XGB ...
Model XGB, Actual: [0 1], [77 29], Predictions: [0 1], [55 51]
Grid search for SVM ...
Grid search for LGB ...
Grid search for RF ...


One or more of the test scores are non-finite: [0.53782773 0.54223546 0.55012381 0.54107853 0.55089354 0.54732863
 0.55405745 0.55937442 0.55398444 0.55878068 0.51587784 0.54071901
 0.54338744 0.54191068 0.54396401 0.55237407 0.5626049  0.56857151
 0.55730307 0.55580339 0.53628243 0.54412048 0.54478981 0.54598449
 0.55325809        nan        nan        nan        nan        nan
        nan        nan        nan        nan        nan        nan
        nan        nan        nan        nan        nan        nan
        nan        nan        nan        nan        nan        nan
        nan        nan]


Grid search for XGB ...
SVM: {'C': 1, 'gamma': 0.001, 'kernel': 'rbf'}
LGB: {'max_depth': 5, 'metric': 'mean_absolute_error', 'num_leaves': 10, 'objective': 'binary'}
RF: {'max_features': 'sqrt', 'min_samples_split': 6, 'n_estimators': 30, 'random_state': 16}
XGB: {'eval_metric': 'error', 'learning_rate': 0.3, 'max_depth': 5, 'n_estimators': 20, 'objective': 'binary:logistic', 'random_state': 16, 'use_label_encoder': False}
Training SVM ...
Model SVM, Actual: [0 1], [46 68], Predictions: [0], [114]
coef_ only available for SVC with linear kernel
Training LGB ...
Model LGB, Actual: [0 1], [46 68], Predictions: [0 1], [86 28]
Training RF ...
Model RF, Actual: [0 1], [46 68], Predictions: [0 1], [85 29]
Training XGB ...
Model XGB, Actual: [0 1], [46 68], Predictions: [0 1], [80 34]
Model evaluation metrics for SVM:
	Accuracy: 0.5254237288135594
	Precision: 0.375
	Recall: 0.6
	F1-score: 0.4615384615384615
	AUC score: 0.5435897435897437
----------------------------------------
	Accuracy: 0.

In [43]:
# ENSEMBLE
importlib.reload(train)
importlib.reload(dr)
importlib.reload(dt)

from scipy.stats import mode
from sklearn.ensemble import VotingClassifier
from sklearn.metrics import f1_score, precision_score, recall_score, roc_auc_score

random.seed(29)

def train_predict_ensemble(ensemble_models, x_train, y_train, x_test, y_test, features, type="majority_vote", weights=None):
    # y_preds = []
    # for model_name in ensemble_models:
    #     x_test_temp = x_test.loc[:, features[model_name]]
    #     y_pred = ensemble_models[model_name].predict(x_test_temp)
    #     y_preds.append(y_pred)
    # if type == "majority_vote":
    #     y_preds = mode(y_preds, axis=0)[0]
    #     y_preds = np.reshape(y_preds, (y_preds.shape[1], 1))
    # elif type == "weighted_avg":
    #     if weights is None:
    #         weights = [1 for _ in y_preds]
    #     for i in range(len(y_preds)):
    #         y_preds[i] = y_preds[i]*weights[i]
    #     y_preds = np.rint(np.mean(y_preds, axis=0))

    features = list(features.values())[0]
    ensemble_models = [(key, ensemble_models[key]) for key in ensemble_models.keys()]
    ensemble = VotingClassifier(estimators=ensemble_models, voting='soft', weights=weights)
    ensemble.fit(x_train.loc[:, features], y_train)
    y_preds = ensemble.predict(x_test.loc[:, features])

    return y_preds


test_size = 0.15
folds = 5
ensemble_models = [
    # "SVM", 
    "LGB",
    "RF",
    "XGB"
]
voting_type = "weighted_avg"
weights = [
    1.2,
    1.0,
    0.8
]

cv, x_train, y_train, x_test, y_test = train.kfold_train_test_split(x, y, test_size, is_resample=False, folds=folds, random_seed=19)
y_train = y_train.loc[:, "label"]
y_test = y_test.loc[:, "label"]

estimators = {name: best_models[name] for name in ensemble_models}
y_pred = train_predict_ensemble(estimators, x_train, y_train, x_test, y_test, features, type=voting_type, weights=weights)

acc = accuracy_score(y_test, y_pred)
p = precision_score(y_test, y_pred, zero_division=0)
r = recall_score(y_test, y_pred, zero_division=0)
f1 = f1_score(y_test, y_pred, zero_division=0)
auc = roc_auc_score(y_test, y_pred)
print(f"\tAccuracy: {acc}\n\tPrecision: {p}\n\tRecall: {r}\n\tF1-score: {f1}\n\tAUC score: {auc}\n" + "-"*40)

	Accuracy: 0.6442953020134228
	Precision: 0.7333333333333333
	Recall: 0.44594594594594594
	F1-score: 0.5546218487394958
	AUC score: 0.642972972972973
----------------------------------------
